In [ ]:
import os
import pandas as pd
from itertools import islice
import torch
from torch.utils.data import DataLoader
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parents[1]))
from utils.text_metrics import evaluate_all_metrics, save_metrics_to_json
from utils.train_comparison import *
from utils.processing import image_transform
from utils.models.gpt_models import DinoGPTCaptioner, DinoGPT2Captioner
from utils.data.chexpert_dataset import CheXpertDataset
from utils.data.padchest_dataset import PadChestGRDataset

# Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

root_dir = "../../../Datasets/PadChest-GR/PadChest_GR"
json_file = "../../../Datasets/PadChest-GR/grounded_reports_20240819.json"
csv_path = "../../../Datasets/PadChest-GR/master_table.csv"

df = pd.read_csv(csv_path)

df_train = df[df['split'] == 'train']
df_validation = df[df['split'] == 'validation']
df_test = df[df['split'] == 'test']


IMG_SIZE = 224
MAX_LEN = 64
NUM_BATCH = 8

tf = image_transform(img_size=IMG_SIZE)

ds_train = PadChestGRDataset(
        dataframe=df_train,
        root_dir=root_dir,
        json_file=json_file,
        max_txt_len=MAX_LEN,
        image_size=IMG_SIZE,
        normalize=True,
        transform=None,
        return_paths=False,
        sentence_key="sentence_en",
    )

ds_valid = PadChestGRDataset(
        dataframe=df_validation,
        root_dir=root_dir,
        json_file=json_file,
        max_txt_len=MAX_LEN,
        image_size=IMG_SIZE,
        normalize=True,
        transform=None,
        return_paths=False,
        sentence_key="sentence_en",
    )

ds_test = PadChestGRDataset(
        dataframe=df_test,
        root_dir=root_dir,
        json_file=json_file,
        max_txt_len=MAX_LEN,
        image_size=IMG_SIZE,
        normalize=True,
        transform=None,
        return_paths=False,
        sentence_key="sentence_en",
    )

tokenizer = build_tokenizer_from_labels()
pad_id = tokenizer.pad_token_id
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id
collate_fn = CaptionCollate(tokenizer, pad_id)

train_loader = DataLoader(ds_train, batch_size=NUM_BATCH, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(ds_valid, batch_size=NUM_BATCH, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(ds_test, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)

Using device: cuda


# Model

In [3]:
# DINO ViT-S/16 hidden size is 384 
EMBEDDING_D_IMG = 384
N_PREFIX = (IMG_SIZE // 16) ** 2  # number of visual prefix tokens (including CLS)

model = DinoGPTCaptioner(
    vocab_size=tokenizer.vocab_size,
    d_img=EMBEDDING_D_IMG,
    pad_id=pad_id,
    d_model=512,
    n_layer=8,
    n_head=8,
    n_prefix=N_PREFIX,           # number of visual prefix tokens
    max_seq_len=256,
    dino_model_id="facebook/dinov3-vits16-pretrain-lvd1689m",
    freeze_dino=True,
).to(device)

# Train Parameters

In [4]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4, weight_decay=1e-2
)
loss = sequence_ce_loss
NUM_EPOCHS = 100
BATCHES_PER_EPOCH = 10

# Training

In [5]:
for epoch in range(NUM_EPOCHS):
    slice_train_loader = islice(train_loader, BATCHES_PER_EPOCH)
    slice_valid_loader = islice(valid_loader, BATCHES_PER_EPOCH)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 10/10 [00:07<00:00,  1.26it/s]


Epoch 1: Train Loss=9.6992, PPL=20625.72 | Val Loss=8.5942, Val PPL=5425.54


Evaluating: 100%|██████████| 10/10 [00:06<00:00,  1.49it/s]


Epoch 2: Train Loss=7.9904, PPL=3372.07 | Val Loss=7.2902, Val PPL=1497.76


Evaluating: 100%|██████████| 10/10 [00:06<00:00,  1.52it/s]


...

Epoch 98: Train Loss=2.7364, PPL=15.79 | Val Loss=3.3198, Val PPL=28.06


Evaluating: 100%|██████████| 10/10 [00:06<00:00,  1.44it/s]


Epoch 99: Train Loss=2.7123, PPL=15.26 | Val Loss=3.2018, Val PPL=28.21


Evaluating: 100%|██████████| 10/10 [00:06<00:00,  1.57it/s]

Epoch 100: Train Loss=2.7243, PPL=15.43 | Val Loss=3.0783, Val PPL=22.54


# Test Parameters

In [14]:
BATCHES_PER_TEST = 1
GREEDY_DECODE = False
TEST_MAX_LEN = 512
TEST_TOP_P = 0.7
TEST_TEMPERATURE = 0.7

# Test

In [15]:
slice_test_loader = islice(test_loader, BATCHES_PER_TEST)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=BATCHES_PER_TEST)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.42it/s]

Test Loss=2.9071, Test PPL=18.30


# Test Report Generation

In [ ]:
with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in test_loader:
        pixel_values = pixel_values.to(device)
        gen_ids = model.generate(
            pixel_values=pixel_values,
            bos_id=bos_id, eos_id=eos_id,
            max_new_tokens=TEST_MAX_LEN, top_p=TEST_TOP_P, temperature=TEST_TEMPERATURE, greedy=GREEDY_DECODE
        )
        print("Predictions (first batch):")
        for i in range(gen_ids.size(0)):
            text_gen = tokenizer.decode(gen_ids[i].tolist())
            text_tgt = tokenizer.decode(ids_loader[i].tolist())
            print(f"\nGEN {i+1}:", text_gen)
            print(f"TGT {i+1}:", text_tgt)
            results = evaluate_all_metrics([text_tgt], [text_gen], evaluation_mode="CheXagent")
            for metric, scores in results.items():
                print(f"{metric}: {scores}")

            save_metrics_to_json(results, f"./results/gpt_model_results_{NUM_EPOCHS}_Padchest.json")
        del pixel_values, ids_loader, paths, raw_labels, gen_ids
        torch.cuda.empty_cache()
        break

Predictions (first batch):

GEN 1: no significant findings.
TGT 1: minimal biapical pleural thickening. slight blunting of the posterior left costophrenic angle. no other significant alterations.
Using device: cuda:0
chexbert_f1_weighted: 0.0
chexbert_f1_micro: 0.0
chexbert_f1_macro: 0.0
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.5073857307434082]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_ER: 0.0
rouge_l: [0.21052631578947367]

GEN 2: no significant findings.
TGT 2: minimal biapical pleural thickening. slight blunting of the posterior left costophrenic angle. no other significant alterations.
Using device: cuda:0
chexbert_f1_weighted: 0.0
chexbert_f1_micro: 0.0
chexbert_f1_macro: 0.0
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.5073857307434082]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_ER: 0.0
rouge_l: [0.21052631578947367]

GEN 3: no significant findings.
TGT 3: slight residual atelectasis in the right pulmonary base. minimal blunting of the costop